In [113]:

from nba_api.live.nba.endpoints import scoreboard
from nba_api.live.nba.endpoints import BoxScore

def fetch_finished_games():
    games = scoreboard.ScoreBoard()
    games = games.get_dict()["scoreboard"]["games"]
    game_ids = [game['gameId'] for game in games if game['gameStatusText'] == 'Final' ]
    return game_ids

def fetch_game_results(game_ids):
    res = []
    for game_id in game_ids:
        box_score = BoxScore(game_id=game_id).get_dict()['game']
        away_team = box_score['awayTeam']
        home_team = box_score['homeTeam']

        winner = away_team  if away_team['score'] > home_team['score'] else home_team
        loser = away_team  if away_team['score'] < home_team['score'] else home_team

        res.append((winner, loser, box_score))

    return res

In [ ]:
from pprint import pprint
import os
import shelve
from pathlib import Path

games = fetch_finished_games()
winners = fetch_game_results(games)

winner, loser, box_score = winners[1]
winner['players']

players = winner['players']
best_player = list(filter(lambda a: a['statistics']['points'] > 30, players))[0]

stats = best_player['statistics']

title = f"{best_player['firstName']} {best_player['familyName']} scored {stats['points']} pts, {stats['reboundsTotal']} reb, and {stats['assists']} ast on {stats['fieldGoalsPercentage']*100:0.0f}/{stats['threePointersPercentage']*100:0.0f}/{stats['freeThrowsPercentage']*100:0.0f} shooting splits in a win against the {loser['teamName']}"

print(title)

box_score['gameId']



In [ ]:
# Save the processed Game ID
import shelve

def process_game_id(game_id: str):
    dir = os.environ['NBPYTHONRUNNER_DATADIR'] if 'NBPYTHONRUNNER_DATADIR' in os.environ else  'C:\\Users\\DevelopPC\\Desktop'
    dir_p = Path(dir)
    fp = dir_p.joinpath("nba_games_processed.shelve")

    with shelve.open(fp) as mydict:
        mydict["game_ids"].append(game_id)